<a href="https://colab.research.google.com/github/mohamaderrr/colab_mlflow_daghshub/blob/main/colab_mlflow_dags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

2110848/2110848 [==============================] - 1s 0us/step


In [10]:
from urllib.parse import urlparse
import os

In [11]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'mohamader14'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '0990298bc71f0570dbddc27da966841a2878edbd'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/mohamader14/ng.mlflow'

In [14]:


# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow

mlflow.tensorflow.autolog()
mlflow.set_experiment('object_detection_model')
max_words = 1000
batch_size = 64
epochs = 5

print("Loading data...")
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

num_classes = np.max(y_train) + 1
print(num_classes, "classes")

print("Vectorizing sequence data...")
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
x_test = tokenizer.sequences_to_matrix(x_test, mode="binary")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

print("Convert class vector to binary class matrix (for use with categorical_crossentropy)")
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

print("Building model...")
model = Sequential()
model.add(Dense(256, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
with mlflow.start_run(run_name='secend run'):
  history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
  score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
  print("Test score:", score[0])
  print("Test accuracy:", score[1])
  remote_server_uri="https://dagshub.com/mohamader14/ng.mlflow"
  mlflow.set_tracking_uri(remote_server_uri)

  tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme



2023/10/25 01:10:35 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Loading data...
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)
Building model...
Epoch 1/5
127/127 [==============================] - 2s 12ms/step - loss: 1.6107 - accuracy: 0.6447 - val_loss: 1.1780 - val_accuracy: 0.7397
Epoch 2/5
127/127 [==============================] - 1s 8ms/step - loss: 0.8795 - accuracy: 0.7982 - val_loss: 0.9909 - val_accuracy: 0.7798
Epoch 3/5
127/127 [==============================] - 1s 8ms/step - loss: 0.6311 - accuracy: 0.8557 - val_loss: 0.9144 - val_accuracy: 0.7909
Epoch 4/5
127/127 [==============================] - 11s 86ms/step - loss: 0.4803 - accuracy: 0.8874 - val_loss: 0.8728 - val_accuracy: 0.8031
Epoch 5/5
1/1 [==============================] - 0s 53ms/step


2023/10/25 01:11:21 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.4.16+cuda11.cudnn86) contains a local version label (+cuda11.cudnn86). MLflow logged a pip requirement for this package as 'jaxlib==0.4.16' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


36/36 [==============================] - 0s 4ms/step - loss: 0.8588 - accuracy: 0.7970
Test score: 0.8588223457336426
Test accuracy: 0.796972393989563
